### Precision and Recall - SVD


In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [147]:
df = pd.read_csv("ratings_small.csv")[:100000] 
df.shape

(100000, 4)

In [148]:
df.drop('timestamp', axis=1, inplace=True)
df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [149]:
# shuffling the data
from sklearn.utils import shuffle
df = shuffle(df)

In [150]:
df.head()

,userId,movieId,rating
9416,63,1,5.0
90162,598,2797,4.0
67505,472,1321,4.5
42919,306,2829,3.0
99147,664,69526,4.0


In [151]:
# train - 80% of the data.
num_rows = df.shape[0]
train, test = df.copy(deep = True), df[int(num_rows*0.8):].copy(deep = True)
train[int(num_rows*0.8):]['rating']  = train[int(num_rows*0.8):]['rating']*0
train = shuffle(train, random_state=0)

In [152]:
# creating user-movie rating utility matrix
user_movie_rating =train.groupby(['userId','movieId'])['rating'].max().unstack().fillna(0)
user_movie_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



- Null values replaced  with zeros

In [153]:
np.sum(user_movie_rating.to_numpy())

283747.0

In [154]:
def low_rank_k(rank, u, s, v):
    # SVD 
    u_new = u[:,:rank]
    v_new = v[:rank, :]
    s_new = s[:rank]
    singular_mat = np.diag(s_new)
    low_rank_mat = np.dot(np.dot(u_new, singular_mat), v_new)
    return low_rank_mat

In [155]:
def error(y_hat, y):
    # MSE
    y_hat = np.array(y_hat)
    y = np.array(y)
    MSE_ = (y_hat- y)**2
    MSE = MSE_.sum()
    return MSE

In [156]:
relevance_threshold = 1
relevant_df = df.loc[(df['rating'] >= relevance_threshold)]

In [157]:
print("Number of relevant user item pair is {} out of {}".format(len(relevant_df), len(df)))

Number of relevant user item pair is 98899 out of 100000


In [158]:
u, s, v = np.linalg.svd(user_movie_rating, full_matrices=False)
optimal_k=10
Pred = low_rank_k(optimal_k, u, s, v)
Pred = pd.DataFrame(Pred, columns=user_movie_rating.columns, index= user_movie_rating.index)
y_hat = [Pred.loc[int(test.iloc[i]['userId']), int(test.iloc[i]['movieId'])] 
             for i in range(len(test))] 

In [159]:
def cur(A, k):
  
  A = A.to_numpy()
  # print(A.shape)

  if (k > A.shape[0]) or (k > A.shape[1]):
    print("error: k greater than matrix dimensions.\n")
    return

  R = np.zeros(( k, A.shape[1]))
  C = np.zeros((A.shape[0], k))  
  W = np.zeros((k, k))

  A_square = A**2
  A_sum_square = np.sum(A_square)
    
  #Prob of each row
  prob_row = np.sum(A_square, axis = 1) / A_sum_square
  #Prob of each col
  prob_col =  np.sum(A_square, axis = 0) / A_sum_square
  


  #selecting rows
  r_count = 0
  row_taken_list = []
  row_dup_list = []

  while r_count < k:
    i = np.random.choice(np.arange(A.shape[0]), p = prob_row)
    # d = d + 1
    #if row already taken, increase the count of duplicates
    if i in row_taken_list:
#       print (len(row_dup_list), len(row_taken_list), i)
      row_dup_list[row_taken_list.index(i)] =  row_dup_list[row_taken_list.index(i)] + 1      
    
    #if row not taken, divide it with root of (k*prob of row) and take it
    elif i not in row_taken_list:
      R[r_count, :] = A[i, :] / np.sqrt(prob_row[i]*k)
      r_count = r_count + 1
      row_taken_list.append(i)
      row_dup_list.append(1)
        
  
  #multiplying rows by root of no. of duplicates
  R = np.multiply(R.T, np.sqrt(row_dup_list))
  R = R.T


  #selecting columns
  c_count = 0
  col_taken_list = []
  col_dup_list = []

  while c_count < k:
    i = np.random.choice(np.arange(A.shape[1]), p = prob_col)
    #if column already taken, increase the count of duplicates
    if i in col_taken_list:
      col_dup_list[col_taken_list.index(i)] =  col_dup_list[col_taken_list.index(i)] + 1      
    
    #if column not taken, divide it with root of (k*prob of column) and take it
    elif i not in col_taken_list:
      C[:, c_count] = A[:, i] / np.sqrt(prob_col[i]*k)
      c_count = c_count + 1
      col_taken_list.append(i)
      col_dup_list.append(1)
  
  #multiplying columns by root of no. of duplicates
  C = np.multiply(C, np.sqrt(col_dup_list))

  #intersection matrix W
  for i, I in enumerate(row_taken_list):
    for j, J in enumerate(col_taken_list):
      W[i, j] = A[I, J]

  #Calculating svd decomposition of W
  X, sigma, Y_T = np.linalg.svd(W, full_matrices=False)
  sigma = np.diag(sigma)

  #Taking pseudo-inverse of sigma
  for i in range(k):
    if sigma[i, i] >= 1:
      sigma[i, i] = 1/sigma[i, i]
    else:
      sigma[i, i] = 0

  #Calculating U
  U = np.dot(Y_T.T, np.dot(np.dot(sigma, sigma), X.T))

  return C, U, R

In [145]:

def CUR_best(A, k, N = 10):
  c, u, r  = cur(A, k)
  err = error(A, c@u@r)
  for i in range(N):
    ctmp, utmp, rtmp  = cur(A, k)
    err_temp = error(A, ctmp@utmp@rtmp)
    Pred_my_new = np.dot(np.dot(ctmp, utmp), rtmp)
    Pred_my_new = pd.DataFrame(Pred_my_new, columns=A.columns, index= A.index)
    y_hat = [Pred_my_new.loc[int(test.iloc[i]['userId']), int(test.iloc[i]['movieId'])] 
                  for i in range(len(test))] 
    err_temp = error(y_hat, test['rating'])
    if err_temp < err:
      err = err_temp
      c = ctmp
      u = utmp
      r = rtmp
  return c, u, r, err

errors= [] 
def give_cur_results(A):
    errors_my_new_1 = []
    ks = np.arange(1, 650, 20)
    for k in ks:
      c,u,r, err = CUR_best(A, k)
      errors_my_new_1.append(err)
    return errors_my_new_1
    
errors=give_cur_results(user_movie_rating)

In [ ]:
#plotting errors_pip
plt.yscale("log")
plt.plot(errors)

plt.figure()

In [ ]:
errors

In [160]:
  %%time
  A = user_movie_rating
  k  = 441
  y_hat = []
  c, u, r  = cur(A, k)
  err = error(A, c@u@r)
  for i in range(0, 1):
    ctmp, utmp, rtmp  = cur(A, k)
    err_temp = error(A, ctmp@utmp@rtmp)
    Pred_my_new = np.dot(np.dot(ctmp, utmp), rtmp)
    Pred_my_new = pd.DataFrame(Pred_my_new, columns=A.columns, index= A.index)
    y_hat_i = [Pred_my_new.loc[int(test.iloc[i]['userId']), int(test.iloc[i]['movieId'])] 
                  for i in range(len(test))] 
    err_temp = error(y_hat_i, test['rating'])
    if err_temp < err:
      err = err_temp
      c = ctmp
      u = utmp
      r = rtmp
      y_hat = y_hat_i

CPU times: user 8.26 s, sys: 394 ms, total: 8.65 s
Wall time: 7.67 s


In [161]:
test['prediction'] = y_hat
test.head()

,userId,movieId,rating,prediction
19291,128,4977,3.0,1.458386
10417,73,1199,4.5,-28.826666
79025,547,2441,3.0,1.035142
15241,99,3160,5.0,0.182133
66654,468,8609,4.5,0.000000


In [162]:
from collections import Counter
user_freq_dict = Counter(test['userId'].tolist())
max_voting = max(user_freq_dict.values())
for i in user_freq_dict:
    if user_freq_dict[i] == max_voting:
        user_choice =i

In [163]:


one_user= test.loc[(test['userId'] == 17)].sort_values('rating', 
                      inplace=False, 
                      ascending=False)
one_user.head()

,userId,movieId,rating,prediction
2773,17,1246,5.0,-10.323133
2712,17,318,5.0,-1.622938
3045,17,31410,5.0,-6.433177
2698,17,111,5.0,-15.287993
3003,17,7371,5.0,-1.140745


### Recall and Precision

In [164]:
relevance_threshold = 2
recommended_movies = set(one_user.loc[(one_user['prediction'] >= relevance_threshold)]['movieId'].tolist())
relavent_movies = set(one_user.loc[(one_user['rating'] >= relevance_threshold)]['movieId'].tolist())
len(relavent_movies), len(recommended_movies)

(67, 19)

In [165]:
recommended_movies_intersection_relavant_movies = [i for i in recommended_movies if i in relavent_movies]
recommended_movies_intersection_relavant_movies;

In [166]:
len(recommended_movies), len(recommended_movies_intersection_relavant_movies), len(relavent_movies)

(19, 16, 67)

In [167]:
recall = len(recommended_movies_intersection_relavant_movies)/len(relavent_movies)
precision = len(recommended_movies_intersection_relavant_movies)/len(recommended_movies)

In [168]:
precision, recall

(0.8421052631578947, 0.23880597014925373)

In [169]:
f_score = 2*precision*recall/(precision+recall)
f_score

0.37209302325581395

### Recall@k and Precision@k
- recall and precision on top k predictions

In [170]:
k=18
top_k_predictions_df = one_user.sort_values('prediction', ascending=False)[:k]
top_k_predictions_df.head()

,userId,movieId,rating,prediction
2709,17,293,4.0,40.903232
2805,17,1732,4.0,15.174841
2818,17,2028,3.5,9.876554
2846,17,2657,3.0,9.738038
2788,17,1411,5.0,7.569000


In [171]:
relevance_threshold=2
top_k_movies_recommended = set(top_k_predictions_df.loc[(top_k_predictions_df['prediction'] >= relevance_threshold)]['movieId'].tolist())
top_k_movies_relevant = set(top_k_predictions_df.loc[(top_k_predictions_df['rating'] >= relevance_threshold)]['movieId'].tolist())
# top_k_predictions_df['rating']

In [172]:
recommended_movies_intersection_relavant_movies = [i for i in top_k_movies_recommended
                                                   if i in top_k_movies_relevant]
recommended_movies_intersection_relavant_movies

[34048,
 2657,
 4993,
 1411,
 1732,
 293,
 1251,
 6502,
 1673,
 288,
 2028,
 2959,
 5782,
 1175,
 8798]

In [173]:
len(top_k_movies_recommended), len(recommended_movies_intersection_relavant_movies), len(top_k_movies_relevant)

(18, 15, 15)

In [174]:
recall = len(recommended_movies_intersection_relavant_movies)/len(top_k_movies_relevant)
precision = len(recommended_movies_intersection_relavant_movies)/len(top_k_movies_recommended)

In [175]:
precision, recall

(0.8333333333333334, 1.0)

In [176]:
f_score = 2*precision*recall/(precision+recall)
f_score

0.9090909090909091